In [24]:
#  데이터 처리 모듈
import pandas as pd
from sklearn.impute import KNNImputer
# 행렬 등 통계 연산 모듈
import numpy as np
# 지수형 표기법 e를 연속형 변환
pd.options.display.float_format = '{:.4f}'.format
# 타입 어노테이션(Any, Sequence 등의 메서드 활용)
from typing import *
import os

# 구글 드라이브 마운트
from google.colab import drive
drive.mount("/content/drive")
import os

# 조합 찾는 반복문 메서드
from itertools import permutations
from itertools import combinations

# 데이터 시각화
import matplotlib.pyplot as plt
# from matplotlib.pyplot import
import matplotlib.font_manager as fm
import missingno as msno
import seaborn as sns

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
# apt-get : 패키지 설치 명령어
# -qq : 에러외의 메세지 숨기는 옵션키
!apt-get -qq install fonts-nanum

fe = fm.FontEntry(
    fname=r"/usr/share/fonts/truetype/nanum/NanumGothic.ttf",
    # ttf 파일이 저장되어 있는 경로
    name='NanumGothic')
# 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)
# Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 18, 'font.family': 'NanumGothic'})
# 폰트 설정

In [26]:
class LoadGoogleDriveData():
  def __init__(self, data = None):
    self.data = data

  def loadData(self, file_path: str, file_name_extension,
               columnTF: bool, unicode: str) -> pd.DataFrame():
    self.data = pd.read_csv(os.path.join(file_path + file_name_extension),
                            index_col = columnTF,
                            na_values = "NaN",
                            encoding = unicode)
    return self.data

  # 용량이 큰 csv 파일 읽어오기(fopen - fread와 유사한 방식)
  def loadDataWithChunking(self, file_path: str, file_name_extension,
                           chunking_row_num: int, columnTF: bool, unicode: str) -> pd.DataFrame():
    chunkdata = pd.read_csv(os.path.join(file_path + file_name_extension),
                            chunksize = chunking_row_num,
                            index_col = columnTF,
                            na_values = "NaN",
                            encoding = unicode)
    self.data = list(chunkdata)
    return self.data


In [27]:
mountInstance = LoadGoogleDriveData()

### 1.오아시스 비즈니스(필지고유번호 & 법정동읍면동코드 & 외식업종)  데이터

In [28]:
CONTEST_2023 = mountInstance.loadDataWithChunking(
    file_path = "/content/drive/MyDrive/2023BigContest/data/contest data/", file_name_extension = "소상공인 매출등급 예상 대상 필지.csv",
    unicode = "cp949", chunking_row_num = 10**5, columnTF = False)

In [29]:
CONTEST_2022 = mountInstance.loadDataWithChunking(
    file_path = "/content/drive/MyDrive/2023BigContest/data/contest data/", file_name_extension = "필지단위 소상공인 매출등급 정보.csv",
    unicode = "cp949", chunking_row_num = 10**5, columnTF = False)

### 2. KCD(상권코드 & 외식업종 중분류) 소상공인 신용데이터

In [30]:
KCD_2022_2023 = mountInstance.loadData(
    file_path = "/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/", file_name_extension = "SOHO_MERGE_DATA_LIST.csv",
    unicode = "utf-8", columnTF = False)

### 3. 행정동코드 - 상권코드 매핑 데이터

In [71]:
MARKET_LOC_MAP = mountInstance.loadData(
    file_path = "/content/drive/MyDrive/2023BigContest/data/",
    file_name_extension = "market_loc_map.csv",
    unicode = "euc-kr", columnTF = False)

In [72]:
MARKET_LOC_MAP

,상권_코드_2022,상권_코드_2023,행정동_코드,법정동_코드,STDG_EMD_CD
0,2110008,3110008,11110515,11110101,11110101
1,2110008,3110008,11110515,11110102,11110102
2,2110008,3110008,11110515,11110103,11110103
3,2110008,3110008,11110515,11110104,11110104
4,2110008,3110008,11110515,11110105,11110105
...,...,...,...,...,...
3453,2130102,3130102,11230750,11230110,11230110
3454,2130166,3130166,11380590,11380107,11380107
3455,2130113,3130113,11260690,11260106,11260106
3456,2130124,3130124,11290780,11290138,11290138


### 4. 서울열린데이터광장 상권특성 공공데이터

In [95]:
# 텍스트 파일 n개가 포함된 C(로컬디스크) 내 파일경로 설정
file_path = "/content/drive/MyDrive/2023BigContest/data" + "/" + "상권_2022"
# 파일경로 내 텍스트 파일명을 디렉토리 리스트에 저장
directory_상권_2022 = os.listdir(file_path)
print(directory_상권_2022, end = ", ")

data_상권특성_2022: List = [
    (directory_상권_2022[index].split("(")[1]).split(")")[0]
    for index in range(len(directory_상권_2022))
]
print("\n", data_상권특성_2022, end = ", ")

['서울시 상권분석서비스(상권영역).csv', '서울시 상권분석서비스(상권배후지-아파트).csv', '서울시 상권분석서비스(상권-소득소비).csv', '서울시 상권분석서비스(상권-아파트).csv', '서울시 상권분석서비스(상권배후지-직장인구).csv', '서울시 상권분석서비스(상권배후지-점포).csv', '서울시 상권분석서비스(상권-점포).csv', '서울시 상권분석서비스(상권배후지-집객시설).csv', '서울시 상권분석서비스(버스 총 승차 승객수 정보).csv', '서울시 상권분석서비스(상권배후지-생활인구).csv', '서울시 상권분석서비스(지하철 총 승차 승객수 정보).csv', '서울시 상권분석서비스(상권-생활인구).csv', '서울시 상권분석서비스(상권-직장인구).csv', '서울시 상권분석서비스(행정동별 상권변화지표).csv', '서울시 상권분석서비스(상권배후지-소득소비).csv'], 
 ['상권영역', '상권배후지-아파트', '상권-소득소비', '상권-아파트', '상권배후지-직장인구', '상권배후지-점포', '상권-점포', '상

In [96]:
# 텍스트 파일 n개가 포함된 C(로컬디스크) 내 파일경로 설정
file_path = "/content/drive/MyDrive/2023BigContest/data" + "/" + "상권_2023"
# 파일경로 내 텍스트 파일명을 디렉토리 리스트에 저장
directory_상권_2023 = os.listdir(file_path)
print(directory_상권_2023, end = ",")

data_상권특성_2023: List = [
    (directory_상권_2023[index].split("(")[1]).split(")")[0]
    for index in range(len(directory_상권_2023))
]
print("\n", data_상권특성_2023, end = ", ")

['서울시 상권분석서비스(상권배후지-점포)_2023.csv', '서울시 상권분석서비스(상권-점포)_2023.csv', '서울시 상권분석서비스(행정동별 상권변화지표)_2023.csv', '서울시 상권분석서비스(상권배후지-직장인구)_2023.csv', '서울시 상권분석서비스(상권-직장인구)_2023.csv', '서울시 상권분석서비스(상권-아파트)_2023.csv', '서울시 상권분석서비스(상권배후지-아파트)_2023.csv', '서울시 상권분석서비스(버스 총 승차 승객수 정보)_2023.csv', '서울시 상권분석서비스(지하철 총 승차 승객수 정보)_2023.csv', '서울시 상권분석서비스(상권배후지-생활인구)_2023.csv', '서울시 상권분석서비스(상권-생활인구)_2023.csv', '서울시 상권분석서비스(상권-소득소비)_2023.csv', '서울시 상권분석서비스(상권배후지-집객시설)_2023.csv'],
 ['상권배후지-점포', '상권-점포', '행정동별 상권변화지표', '상권배후지-직장인구', '상권-직장인구', '상권-아파트', '상권배후지-아파트', '버스 총 승차 승객수 정

In [98]:
# 상권특성_2022 및 상권특성_2023 데이터의 교집합
print(np.intersect1d(data_상권특성_2022, data_상권특성_2023))
# 상권특성_2022 및 상권특성_2023 데이터의 여집합

['버스 총 승차 승객수 정보' '상권-생활인구' '상권-소득소비'
 '상권-아파트' '상권-점포' '상권-직장인구'
 '상권배후지-생활인구' '상권배후지-아파트' '상권배후지-점포'
 '상권배후지-직장인구' '상권배후지-집객시설'
 '지하철 총 승차 승객수 정보' '행정동별 상권변화지표']


In [100]:
print(np.setxor1d(data_상권특성_2022, data_상권특성_2023))

['상권배후지-소득소비' '상권영역']


In [101]:
directory_상권_2022.remove("서울시 상권분석서비스(상권배후지-소득소비).csv")
data_상권특성_2022.remove("상권배후지-소득소비")

In [103]:
MARKET_2022_LIST = []
for index, data in zip(list(range(len(directory_상권_2022))), directory_상권_2022):
  MARKET_2022 = mountInstance.loadData(
    file_path = "/content/drive/MyDrive/2023BigContest/data/상권_2022/",
    file_name_extension = data,
    unicode = "euc-kr", columnTF = False)
  MARKET_2022_LIST.append(MARKET_2022)
  MARKET_2022 = 0

In [ ]:
MARKET_2023_LIST = []
directory_상권_2023: List = [item.split(".csv")[0] + "_2023.csv" for item in directory_상권_2022[1:]]

for index, data in zip(list(range(len(directory_상권_2023))), directory_상권_2023):
  MARKET_2023 = mountInstance.loadData(
    file_path = "/content/drive/MyDrive/2023BigContest/data/상권_2023/",
    file_name_extension = data,
    unicode = "euc-kr", columnTF = False)
  MARKET_2023_LIST.append(MARKET_2023)
  MARKET_2023 = 0

In [111]:
display(directory_상권_2022)
# MARKET_2022_LIST[0]

['서울시 상권분석서비스(상권영역).csv',
 '서울시 상권분석서비스(상권배후지-아파트).csv',
 '서울시 상권분석서비스(상권-소득소비).csv',
 '서울시 상권분석서비스(상권-아파트).csv',
 '서울시 상권분석서비스(상권배후지-직장인구).csv',
 '서울시 상권분석서비스(상권배후지-점포).csv',
 '서울시 상권분석서비스(상권-점포).csv',
 '서울시 상권분석서비스(상권배후지-집객시설).csv',
 '서울시 상권분석서비스(버스 총 승차 승객수 정보).csv',
 '서울시 상권분석서비스(상권배후지-생활인구).csv',
 '서울시 상권분석서비스(지하철 총 승차 승객수 정보).csv',
 '서울시 상권분석서비스(상권-생활인구).csv',
 '서울시 상권분석서비스(상권-직장인구).csv',
 '서울시 상권분석서비스(행정동별 상권변화지표).csv']

In [113]:
# display(directory_상권_2023)
# MARKET_2023_LIST[6]

### 5.분기별 상권특성 공공데이터 전처리

#### 5.(1) 상권_소득소비

  MARKET_2022_LIST[2]

  MARKET_2023_LIST[1]

  ```
  SELECT
    AVERAGE(seoul.상권특성.교육_지출_총금액),
    AVERAGE(seoul.상권특성.교통_지출_총금액),
    AVERAGE(seoul.상권특성.생활용품_지출_총금액),
    AVERAGE(seoul.상권특성.문화_지출_총금액),
    AVERAGE(seoul.상권특성.유흥_지출_총금액),
    AVERAGE(seoul.상권특성.의료비_지출_총금액),
    AVERAGE(seoul.상권특성.의류_신발_지출_총금액),
    AVERAGE(seoul.상권특성.식료품_지출_총금액),
    AVERAGE(seoul.상권특성.여가_지출_총금액),
    COUNT(*),
    SUM(kcd.meta-info.is_franchise)
  FROM kcd.meta-info
  JOIN seoul.상권특성
  ON kcd.meta-info.trdar_no = seoul.상권특성.상권_코드
  WHERE kcd.meta-info.class_1_name = "외식업"
  GROUP BY seoul.상권특성.기준_년_코드, seoul.상권특성.기준_분기_코드, kcd.meta-info.class_2_name
  ```

In [120]:
MARKET_2022_LIST[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6520 entries, 0 to 6519
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   기준_년_코드       6520 non-null   int64  
 1   기준_분기_코드      6520 non-null   int64  
 2   상권_구분_코드      6520 non-null   object 
 3   상권_구분_코드_명    6520 non-null   object 
 4   상권_코드         6520 non-null   int64  
 5   상권_코드_명       6520 non-null   object 
 6   월_평균_소득_금액    6480 non-null   float64
 7   소득_구간_코드      6480 non-null   float64
 8   지출_총금액        6388 non-null   float64
 9   식료품_지출_총금액    6388 non-null   float64
 10  의류_신발_지출_총금액  6388 non-null   float64
 11  생활용품_지출_총금액   6388 non-null   float64
 12  의료비_지출_총금액    6388 non-null   float64
 13  교통_지출_총금액     6388 non-null   float64
 14  여가_지출_총금액     6388 non-null   float64
 15  문화_지출_총금액     6388 non-null   float64
 16  교육_지출_총금액     6388 non-null   float64
 17  유흥_지출_총금액     6388 non-null   float64
dtypes: float64(12), int64(3), ob

#### 5.(2) 상권_배후지_아파트
  
  MARKET_2022_LIST[3]

  MARKET_2023_LIST[2]

  ```
  SELECT
    AVERAGE(seoul.상권특성.아파트_단지_수),
    AVERAGE(seoul.상권특성.아파트_가격_1_억_미만_세대_수),
    AVERAGE(seoul.상권특성.아파트_가격_1_억_세대_수),
    AVERAGE(seoul.상권특성.아파트_가격_2_억_세대_수),
    AVERAGE(seoul.상권특성.아파트_가격_3_억_세대_수),
    AVERAGE(seoul.상권특성.아파트_가격_4_억_세대_수),
    AVERAGE(seoul.상권특성.아파트_가격_5_억_세대_수),
    AVERAGE(seoul.상권특성.아파트_가격_6_억_이상_세대_수),
    COUNT(*),
    SUM(kcd.meta-info.is_franchise)
  FROM kcd.meta-info
  JOIN seoul.상권특성
  ON kcd.meta-info.trdar_nm = seoul.상권특성.상권_코드_명
  WHERE kcd.meta-info.class_1_name = "외식업"
  GROUP BY seoul.상권특성.기준_년_코드, seoul.상권특성.기준_분기_코드, kcd.meta-info.class_2_name
  ```

In [122]:
MARKET_2022_LIST[3].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5932 entries, 0 to 5931
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   기준_년_코드             5932 non-null   int64  
 1   기준_분기_코드            5932 non-null   int64  
 2   상권_구분_코드            5932 non-null   object 
 3   상권_구분_코드_명          5932 non-null   object 
 4   상권_코드               5932 non-null   int64  
 5   상권_코드_명             5932 non-null   object 
 6   아파트_단지_수            5932 non-null   int64  
 7   아파트_가격_1_억_미만_세대_수  4720 non-null   float64
 8   아파트_가격_1_억_세대_수     5520 non-null   float64
 9   아파트_가격_2_억_세대_수     5440 non-null   float64
 10  아파트_가격_3_억_세대_수     4232 non-null   float64
 11  아파트_가격_4_억_세대_수     2984 non-null   float64
 12  아파트_가격_5_억_세대_수     2120 non-null   float64
 13  아파트_가격_6_억_이상_세대_수  1776 non-null   float64
dtypes: float64(7), int64(4), object(3)
memory usage: 648.9+ KB


#### 5.(3) 상권배후지_점포

  MARKET_2022_LIST[6]

  MARKET_2023_LIST[5]

  ```
  SELECT
    AVERAGE(seoul.상권특성.개업_점포_수),
    AVERAGE(seoul.상권특성.유사_업종_점포_수),
    AVERAGE(seoul.상권특성.폐업_점포_수),
    AVERAGE(seoul.상권특성.점포_수),
    AVERAGE(seoul.상권특성.프랜차이즈_점포_수),
    STANDARD_DEVIATION_POPULATION(seoul.상권특성.개업_율),
    STANDARD_DEVIATION_POPULATION(seoul.상권특성.폐업_율),
    COUNT(*),
    SUM(kcd.meta-info.is_franchise)
  FROM kcd.meta-info
  JOIN seoul.상권특성
  ON kcd.meta-info.trdar_nm = seoul.상권특성.상권_코드_명
  WHERE kcd.meta-info.class_1_name = "외식업"
  GROUP BY seoul.상권특성.기준_년_코드, seoul.상권특성.기준_분기_코드, kcd.meta-info.class_2_name
  ```

In [125]:
MARKET_2022_LIST[6].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299665 entries, 0 to 299664
Data columns (total 15 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   기준_년_코드      299665 non-null  int64 
 1   기준_분기_코드     299665 non-null  int64 
 2   상권_구분_코드     299665 non-null  object
 3   상권_구분_코드_명   299665 non-null  object
 4   상권_코드        299665 non-null  int64 
 5   상권_코드_명      299665 non-null  object
 6   서비스_업종_코드    299665 non-null  object
 7   서비스_업종_코드_명  299665 non-null  object
 8   점포_수         299665 non-null  int64 
 9   유사_업종_점포_수   299665 non-null  int64 
 10  개업_율         299665 non-null  int64 
 11  개업_점포_수      299665 non-null  int64 
 12  폐업_률         299665 non-null  int64 
 13  폐업_점포_수      299665 non-null  int64 
 14  프랜차이즈_점포_수   299665 non-null  int64 
dtypes: int64(10), object(5)
memory usage: 34.3+ MB


#### 5.(4) 상권배후지_집객시설

  MARKET_2022_LIST[7]

  MARKET_2023_LIST[6]
    
  ```
  SELECT
    AVERAGE(seoul.상권특성.집객시설_수),
    AVERAGE(seoul.상권특성.관공서_수),
    AVERAGE(seoul.상권특성.은행_수),
    AVERAGE(seoul.상권특성.종합병원_수),
    AVERAGE(seoul.상권특성.일반_병원_수),
    AVERAGE(seoul.상권특성.약국_수),
    AVERAGE(seoul.상권특성.유치원_수),
    AVERAGE(seoul.상권특성.초등학교_수),
    AVERAGE(seoul.상권특성.고등학교_수),
    AVERAGE(seoul.상권특성.대학교_수),
    AVERAGE(seoul.상권특성.백화점_수),
    AVERAGE(seoul.상권특성.슈퍼마켓_수),
    AVERAGE(seoul.상권특성.숙박_시설_수),AVERAGE(seoul.상권특성.버스_터미널_수),
    AVERAGE(seoul.상권특성.지하철_역_수),
    AVERAGE(seoul.상권특성.버스_정거장_수),
    COUNT(*),
    SUM(kcd.meta-info.is_franchise)
  FROM kcd.meta-info
  JOIN seoul.상권특성
  ON kcd.meta-info.trdar_no = seoul.상권특성.상권_코드
  WHERE kcd.meta-info.class_1_name = "외식업"
  GROUP BY seoul.상권특성.기준_년_코드, seoul.상권특성.기준_분기_코드, kcd.meta-info.class_2_name
  ```

In [126]:
MARKET_2022_LIST[7].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4360 entries, 0 to 4359
Data columns (total 26 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   기준_년_코드     4360 non-null   int64  
 1   기준_분기_코드    4360 non-null   int64  
 2   상권_구분_코드    4360 non-null   object 
 3   상권_구분_코드_명  4360 non-null   object 
 4   상권_코드       4360 non-null   int64  
 5   상권_코드_명     4360 non-null   object 
 6   집객시설_수      4360 non-null   int64  
 7   관공서_수       4160 non-null   float64
 8   은행_수        3684 non-null   float64
 9   종합병원_수      540 non-null    float64
 10  일반_병원_수     1472 non-null   float64
 11  약국_수        4204 non-null   float64
 12  유치원_수       2376 non-null   float64
 13  초등학교_수      3336 non-null   float64
 14  중학교_수       2180 non-null   float64
 15  고등학교_수      1836 non-null   float64
 16  대학교_수       1276 non-null   float64
 17  백화점_수       52 non-null     float64
 18  슈퍼마켓_수      904 non-null    float64
 19  극장_수        8 non-null     

In [130]:
MARKET_2022_LIST[9].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4360 entries, 0 to 4359
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   기준_년_코드           4360 non-null   int64 
 1   기준_분기_코드          4360 non-null   int64 
 2   상권_구분_코드          4360 non-null   object
 3   상권_구분_코드_명        4360 non-null   object
 4   상권_코드             4360 non-null   int64 
 5   총_생활인구_수          4360 non-null   int64 
 6   남성_생활인구_수         4360 non-null   int64 
 7   여성_생활인구_수         4360 non-null   int64 
 8   연령대_10_생활인구_수     4360 non-null   int64 
 9   연령대_20_생활인구_수     4360 non-null   int64 
 10  연령대_30_생활인구_수     4360 non-null   int64 
 11  연령대_40_생활인구_수     4360 non-null   int64 
 12  연령대_50_생활인구_수     4360 non-null   int64 
 13  연령대_60_이상_생활인구_수  4360 non-null   int64 
 14  시간대_1_생활인구_수      4360 non-null   int64 
 15  시간대_2_생활인구_수      4360 non-null   int64 
 16  시간대_3_생활인구_수      4360 non-null   int64 
 17  시간대_4_생활인구_수  

#### 5.(5) 상권_상주인구(상권-생활인구)

  MARKET_2022_LIST[11]

  MARKET_2022_LIST[10]

  ```
  SELECT
    AVERAGE(seoul.상권특성.연령대_10_상주인구_수),
    AVERAGE(seoul.상권특성.연령대_20_상주인구_수),
    AVERAGE(seoul.상권특성.연령대_30_상주인구_수),
    AVERAGE(seoul.상권특성.연령대_40_상주인구_수),
    AVERAGE(seoul.상권특성.연령대_50_상주인구_수),
    AVERAGE(seoul.상권특성.연령대_60_이상_상주인구_수),
    AVERAGE(seoul.상권특성.총_가구_수),
    AVERAGE(seoul.상권특성.비_아파트_가구_수),
    AVERAGE(seoul.상권특성.아파트_가구_수),
    AVERAGE(seoul.상권특성.총_상주인구_수),
    AVERAGE(seoul.상권특성.남성_상주인구_수),
    AVERAGE(seoul.상권특성.여성_상주인구_수),
    COUNT(*),
    SUM(kcd.meta-info.is_franchise)
  FROM kcd.meta-info
  JOIN seoul.상권특성
  ON kcd.meta-info.trdar_no = seoul.상권특성.상권_코드
  WHERE kcd.meta-info.class_1_name = "외식업"
  GROUP BY seoul.상권특성.기준_년_코드, seoul.상권특성.기준_분기_코드, kcd.meta-info.class_2_name
  ```

In [139]:
MARKET_2022_LIST[11].rename(columns = {" 상권_구분_코드_명":"상권_구분_코드_명"}, inplace = True)
MARKET_2022_LIST[10].rename(columns = {" 상권_구분_코드_명":"상권_구분_코드_명"}, inplace = True)

In [140]:
MARKET_2022_LIST[11].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6674 entries, 0 to 6673
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   기준_년_코드           6674 non-null   int64 
 1   기준_분기_코드          6674 non-null   int64 
 2   상권_구분_코드          6674 non-null   object
 3   상권_구분_코드_명        6674 non-null   object
 4   상권_코드             6674 non-null   int64 
 5   상권_코드_명           6674 non-null   object
 6   총_생활인구_수          6674 non-null   int64 
 7   남성_생활인구_수         6674 non-null   int64 
 8   여성_생활인구_수         6674 non-null   int64 
 9   연령대_10_생활인구_수     6674 non-null   int64 
 10  연령대_20_생활인구_수     6674 non-null   int64 
 11  연령대_30_생활인구_수     6674 non-null   int64 
 12  연령대_40_생활인구_수     6674 non-null   int64 
 13  연령대_50_생활인구_수     6674 non-null   int64 
 14  연령대_60_이상_생활인구_수  6674 non-null   int64 
 15  시간대_1_생활인구_수      6674 non-null   int64 
 16  시간대_2_생활인구_수      6674 non-null   int64 
 17  시간대_3_생활인구_수  

#### 5.(6) 상권_점포(행정동별 상권변화지표)

  ['상권축소', '정체', '다이나믹', '상권확장']
  
  MARKET_2022_LIST[13]

  MARKET_2023_LIST[12]

  ```
  SELECT
    AVERAGE(seoul.상권특성.개업_점포_수),
    AVERAGE(seoul.상권특성.유사_업종_점포_수),
    AVERAGE(seoul.상권특성.폐업_점포_수),
    AVERAGE(seoul.상권특성.점포_수),
    AVERAGE(seoul.상권특성.프랜차이즈_점포_수),
    STANDARD_DEVIATION_POPULATION(seoul.상권특성.개업_율),
    STANDARD_DEVIATION_POPULATION(seoul.상권특성.폐업_율),
    COUNT(*),
    SUM(kcd.meta-info.is_franchise)
  FROM kcd.meta-info
  JOIN seoul.상권특성
  ON kcd.meta-info.trdar_nm = seoul.상권특성.상권_코드_명
  WHERE kcd.meta-info.class_1_name = "외식업"
  GROUP BY seoul.상권특성.기준_년_코드, seoul.상권특성.기준_분기_코드, kcd.meta-info.class_2_name
  ```

In [129]:
MARKET_2022_LIST[13].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1696 entries, 0 to 1695
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   기준_년_코드         1696 non-null   int64 
 1   기준_분기_코드        1696 non-null   int64 
 2   행정동_코드          1696 non-null   int64 
 3   행정동_코드_명        1696 non-null   object
 4   상권_변화_지표        1696 non-null   object
 5   상권_변화_지표_명      1696 non-null   object
 6   운영_영업_개월_평균     1696 non-null   int64 
 7   폐업_영업_개월_평균     1696 non-null   int64 
 8   서울_운영_영업_개월_평균  1696 non-null   int64 
 9   서울_폐업_영업_개월_평균  1696 non-null   int64 
dtypes: int64(7), object(3)
memory usage: 132.6+ KB


___

In [141]:
for index in [2, 3, 6, 7, 11]:
  if index == 2:
    MERGE_2022_DATA = MARKET_2022_LIST[index]
  else:
    MERGE_2022_DATA = pd.merge(
        left = MERGE_2022_DATA, right = MARKET_2022_LIST[index],
        on = ["기준_년_코드", "기준_분기_코드", "상권_구분_코드", "상권_구분_코드_명", "상권_코드", "상권_코드_명"],
        how = "inner")

In [148]:
MARKET_2022_LIST[0].drop(["형태정보"], axis = 1, inplace = True)

In [150]:
MARKET_LOC_MAP.info()
MARKET_LOC_MAP

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3458 entries, 0 to 3457
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   상권_코드_2022   3458 non-null   int64
 1   상권_코드_2023   3458 non-null   int64
 2   행정동_코드       3458 non-null   int64
 3   법정동_코드       3458 non-null   int64
 4   STDG_EMD_CD  3458 non-null   int64
dtypes: int64(5)
memory usage: 135.2 KB


,상권_코드_2022,상권_코드_2023,행정동_코드,법정동_코드,STDG_EMD_CD
0,2110008,3110008,11110515,11110101,11110101
1,2110008,3110008,11110515,11110102,11110102
2,2110008,3110008,11110515,11110103,11110103
3,2110008,3110008,11110515,11110104,11110104
4,2110008,3110008,11110515,11110105,11110105
...,...,...,...,...,...
3453,2130102,3130102,11230750,11230110,11230110
3454,2130166,3130166,11380590,11380107,11380107
3455,2130113,3130113,11260690,11260106,11260106
3456,2130124,3130124,11290780,11290138,11290138


In [ ]:
MARKET_2022_LIST[13] = pd.merge(
    left = MARKET_LOC_MAP, right = MARKET_2022_LIST[13],
    on = ["행정동_코드"],
    how = "inner")

MERGE_2022_DATA = pd.merge(
        left = MERGE_2022_DATA, right = MARKET_2022_LIST[13],
        on = ["기준_년_코드", "기준_분기_코드", "행정동_코드"],
        how = "inner")

In [143]:
MERGE_2022_DATA.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 178774 entries, 0 to 178773
Data columns (total 77 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   기준_년_코드             178774 non-null  int64  
 1   기준_분기_코드            178774 non-null  int64  
 2   상권_구분_코드            178774 non-null  object 
 3   상권_구분_코드_명          178774 non-null  object 
 4   상권_코드               178774 non-null  int64  
 5   상권_코드_명             178774 non-null  object 
 6   월_평균_소득_금액          178711 non-null  float64
 7   소득_구간_코드            178711 non-null  float64
 8   지출_총금액              178774 non-null  float64
 9   식료품_지출_총금액          178774 non-null  float64
 10  의류_신발_지출_총금액        178774 non-null  float64
 11  생활용품_지출_총금액         178774 non-null  float64
 12  의료비_지출_총금액          178774 non-null  float64
 13  교통_지출_총금액           178774 non-null  float64
 14  여가_지출_총금액           178774 non-null  float64
 15  문화_지출_총금액           178774 non-nul